# Options Configuration: Tuning Your Quantum Jobs (EXAM TESTED!)

> **Exam Weight**: Part of 15% (Section 4) | **Must Master**: ✅✅

## 🧠 Conceptual Overview

### Analogy: Camera Settings

Think of **Options** as the "settings menu" before taking a photo:

| Camera Setting | Options Equivalent | Purpose |
|----------------|-------------------|---------|
| **ISO** | `optimization_level` | Quality vs speed tradeoff |
| **Flash** | `resilience_level` | Compensate for poor conditions (noise) |
| **Burst Mode** | `execution.shots` | How many photos to average |
| **Image Stabilization** | `dynamical_decoupling` | Fight motion blur (decoherence) |

### Why Options Matter

```
Without Options:               With Optimized Options:
┌─────────────────────┐       ┌─────────────────────┐
│ Default settings    │       │ Tuned for your needs│
│ • optimization=2    │       │ • optimization=3    │
│ • resilience=0      │       │ • resilience=1      │
│ • shots=4096        │       │ • shots=8192        │
└─────────────────────┘       └─────────────────────┘
         ↓                             ↓
   Generic results                Optimized results
   ~50% accuracy                  ~85% accuracy
```

## Part 1: Options Basics

In [ ]:
# Options class
print("Options Configuration:")
print("""
from qiskit_ibm_runtime import Options

options = Options()

# Nested namespaces:
# - options.execution.*    (shots, init_qubits)
# - options.simulator.*    (seed_simulator)
# - options.resilience.*   (resilience_level)
# - options.optimization_level (0-3)
""")

print("\n✓ Imported from qiskit_ibm_runtime")
print("✓ Passed to Sampler/Estimator constructor")
print("✓ Can update after creation")

### ⚠️ EXAM TRAP: Default Shots Value

```python
# Default for Options
options.execution.shots = 4096  # ✓ Default

# NOT 1024 (old default)
# NOT 100 or 1000
```

**CRITICAL**: Default is **4096**, not 1024!

## Part 2: Execution Options

In [ ]:
# Execution configuration
print("Execution Options:")
print("""
options = Options()

# Set number of shots
options.execution.shots = 1000

# Initialize qubits to |0>
options.execution.init_qubits = True  # Default True
""")

print("\nKey Execution Parameters:")
print("  - shots: Number of circuit executions (default 4096)")
print("  - init_qubits: Reset qubits to |0> (default True)")

print("\n🎯 EXAM TIP: Access via options.execution.shots NOT options.shots!")

## Part 3: Optimization Level

In [ ]:
# Optimization level
print("Optimization Level:")
print("""
options = Options()
options.optimization_level = 2  # Default

# Levels:
# 0 - No optimization
# 1 - Light optimization (transpile default)
# 2 - Moderate (Options default) ✓
# 3 - Heavy optimization
""")

print("\n⚠️ CRITICAL DIFFERENCE:")
print("  transpile() default: optimization_level=1")
print("  Options() default:   optimization_level=2")
print("\nNOT the same!")
print("""
options = Options()
options.resilience_level = 1

# Resilience Levels:
# 0 - No error mitigation (default)
# 1 - Basic mitigation (recommended)
# 2 - Advanced mitigation (more overhead)
""")

print("\nResilience Methods:")
print("  Level 0: None")
print("  Level 1: Twirled Readout Error Extinction (TREX)")
print("  Level 2: Zero-Noise Extrapolation (ZNE)")

print("\n✓ Higher level = Better accuracy")
print("✓ Higher level = More shots/overhead")
print("✓ Start with level 1 for real hardware")

### ⚠️ EXAM TRAP: Resilience Levels

```python
# CORRECT - resilience_level (singular)
options.resilience_level = 1  # ✓

# WRONG - resilience.level (nested)
options.resilience.level = 1  # ❌
```

**Note**: Top-level attribute, NOT nested like execution.shots!

### Visual: Error Mitigation Effect

```
Without Mitigation (Level 0):       With M3 (Level 1):
    
Ideal: 50% |00⟩, 50% |11⟩         Ideal: 50% |00⟩, 50% |11⟩

Measured:                          Corrected:
|00⟩: ████████ 45%                |00⟩: █████████ 49%
|01⟩: █ 3%   (error!)             |01⟩:  1%
|10⟩: █ 2%   (error!)             |10⟩:  <1%
|11⟩: ████████ 50%                |11⟩: ██████████ 50%

Readout errors visible!            Corrected to near-ideal!
```

### Resilience Level Guide

| Level | Method | Overhead | Use Case |
|-------|--------|----------|----------|
| 0 | None | 0% | Simulator, debugging |
| 1 | M3 (Matrix-free Measurement Mitigation) | ~20% | Most production |
| 2 | ZNE + M3 (Zero-Noise Extrapolation) | ~300% | Critical accuracy |

## Part 5: Simulator Options

In [ ]:
# Simulator configuration
print("Simulator Options:")
print("""
options = Options()

# Set seed for reproducibility
options.simulator.seed_simulator = 42

# Only applies to simulators, ignored by real hardware
""")

print("\n✓ seed_simulator ensures reproducible results")
print("✓ Useful for testing and debugging")
print("✓ Ignored by real backends")

## Part 6: Complete Configuration Pattern

In [ ]:
# Full configuration
print("Complete Options Pattern:")
print("""
from qiskit_ibm_runtime import Sampler, Options

# Create and configure options
options = Options()
options.optimization_level = 3
options.resilience_level = 1
options.execution.shots = 2000
options.simulator.seed_simulator = 42

# Pass to primitive
sampler = Sampler(backend=backend, options=options)

# Run
job = sampler.run([circuit])
result = job.result()
""")

print("\n✓ Configure before passing to primitive")
print("✓ Can also update after: sampler.options.execution.shots = 5000")

### Options Namespace Structure

```
Options()
├── optimization_level (0-3)        [top-level]
├── resilience_level (0-2)          [top-level]
├── execution
│   ├── shots (default 4096)
│   └── init_qubits (default True)
├── simulator
│   └── seed_simulator (optional)
├── transpilation
│   ├── skip_transpilation (False)
│   └── initial_layout ([0,1,2,...])
└── dynamical_decoupling
    ├── enable (True/False)
    └── sequence_type ('XY4', 'XX', etc.)
```

## Part 7: Options Decision Tree (EXAM CRITICAL!)

```
Choose Options based on:

┌─ What's your priority? ──────────────────────┐
│  Speed → optimization_level=1, resilience=0  │
│  Quality → optimization_level=3, resilience=1│
│  Research → optimization_level=3, resilience=2│
└───────────────────────────────────────────────┘

┌─ What hardware? ─────────────────────────────┐
│  Simulator → resilience_level=0              │
│  Real hardware → resilience_level=1          │
│  Noisy hardware → resilience_level=2         │
└───────────────────────────────────────────────┘

┌─ How precise? ───────────────────────────────┐
│  Quick test → shots=1024                     │
│  Standard → shots=4096                       │
│  High precision → shots=8192+                │
└───────────────────────────────────────────────┘
```

### Common Configuration Patterns

```python
# Rapid Prototyping (simulator)
options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = 1024

# Production (real hardware)
options = Options()
options.optimization_level = 3
options.resilience_level = 1
options.execution.shots = 4096
options.dynamical_decoupling.enable = True

# Maximum Quality (research)
options = Options()
options.optimization_level = 3
options.resilience_level = 2
options.execution.shots = 8192
```

**Memory Aid**: "Options = Optimization + Resilience + Shots"

## 📝 Practice Questions

### Question 1: Default Shots

**What is the default value for execution.shots?**

A) 1024  
B) 2048  
C) 4096  
D) 8192

<details>
<summary>Answer</summary>

**C) 4096**

```python
options = Options()
print(options.execution.shots)  # 4096
```

**CRITICAL**: NOT 1024 (old default)!
</details>

---

### Question 2: Optimization Level Default

**What is the default optimization_level in Options()?**

A) 0  
B) 1  
C) 2  
D) 3

<details>
<summary>Answer</summary>

**C) 2**

```python
options = Options()
print(options.optimization_level)  # 2
```

**Note**: Different from transpile() default (1)!
</details>

---

### Question 3: Accessing Shots

**How to access shots configuration?**

A) `options.shots`  
B) `options.execution.shots`  
C) `options.run.shots`  
D) `options.config.shots`

<details>
<summary>Answer</summary>

**B) `options.execution.shots`**

```python
# CORRECT - nested under execution
options.execution.shots = 1000  # ✓

# WRONG - not top-level
options.shots = 1000  # ❌
```
</details>

---

### Question 4: Resilience Level

**What does resilience_level=1 enable?**

A) Circuit optimization  
B) Error mitigation  
C) More shots  
D) Transpilation

<details>
<summary>Answer</summary>

**B) Error mitigation**

Resilience levels:
- 0: No mitigation
- 1: Basic (TREX)
- 2: Advanced (ZNE)

Mitigates readout/gate errors on real hardware.
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Default Values**
   - `execution.shots = 4096` (NOT 1024!)
   - `optimization_level = 2` (NOT 1 like transpile!)
   - `resilience_level = 0` (no mitigation)

2. **Namespace Structure**
   - Top-level: optimization_level, resilience_level
   - Nested execution: shots, init_qubits
   - Nested simulator: seed_simulator

3. **Resilience**
   - Level 0: None
   - Level 1: Basic (TREX)
   - Level 2: Advanced (ZNE)

### Critical Exam Facts

- ✅ Default shots: **4096** (not 1024)
- ✅ Default optimization_level: **2** (not 1)
- ✅ Access: `options.execution.shots` (nested)
- ✅ Resilience: `options.resilience_level` (top-level)
- ✅ Pass to primitive: `Sampler(backend=backend, options=options)`
- ✅ Can update after creation

### Common Traps

- ❌ `options.shots` → ✅ `options.execution.shots`
- ❌ Default 1024 → ✅ Default 4096
- ❌ Both default 1 → ✅ Options=2, transpile=1

### Mnemonic

🧠 **"4096 Shots, Level 2, Resilience 0-1-2!"**

**Next**: Backend Target (V2 API)!